In [26]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [111]:
# Load in your data here
data = pd.read_csv("/home/novo/icr-2025-psych/OCDhc_allFC.csv") #file name here

In [113]:
# Data preprocessing
X = data.drop('1026', axis=1)
y = data['1026']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [114]:
X_train.head

<bound method NDFrame.head of              0          1          2          3          4          5  \
2    88.949141  69.719005  87.931699  90.992738  75.977582  62.116550   
86   80.468150  74.724858  99.725039  92.412112  70.111520  41.526285   
12   66.453976  57.538885  71.938036  65.686140  52.806723  41.372298   
92   86.885283  45.571536  83.573928  74.586587  70.745728  62.620254   
93   77.389677  61.301117  99.789757  93.558635  63.484386  39.678047   
..         ...        ...        ...        ...        ...        ...   
29   56.668081  57.873559  67.443090  73.525440  63.285887  55.895373   
110  79.921022  46.939511  68.489176  76.480141  51.587711  31.996168   
84   51.903003  99.701376  99.778920  78.109412  78.079135  64.240750   
28   81.790022  69.774932  70.340084  68.444542  68.366193  61.398505   
73   82.291907  74.414926  74.393236  69.410711  69.120416  45.828502   

             6          7          8          9  ...       1016       1017  \
2    41.179372 

In [115]:
y_train.head

<bound method NDFrame.head of 2      1
86     0
12     1
92     0
93     0
      ..
29     1
110    0
84     0
28     1
73     0
Name: 1026, Length: 112, dtype: int64>

In [116]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [117]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))  # Dropout helps prevent overfitting
model.add(Dense(units=1, activation='sigmoid'))

/home/novo/miniforge3/envs/neural/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [118]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])


In [123]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=200, batch_size=32)
from tensorflow.keras.metrics import AUC

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6470 - auc: 0.4250 - loss: 0.6546  
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6824 - auc: 0.4690 - loss: 0.6270 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6783 - auc: 0.5927 - loss: 0.6254 
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6324 - auc: 0.4520 - loss: 0.6624 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7085 - auc: 0.5668 - loss: 0.6053 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6991 - auc: 0.6233 - loss: 0.6099 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6929 - auc: 0.4737 - loss: 0.6202 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6939 - auc: 0.5317 - loss: 0.6154 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6876 - auc: 0.4041 - loss: 0.6272 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6647 - auc: 0.5375 - loss: 0.6363 
Epoch 11/200
4/4 ━

In [124]:
y_hat = (model.predict(X_test) > 0.5).astype(int).flatten()
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [125]:
accuracy_score(y_test, y_hat)

0.6206896551724138

In [126]:
from sklearn.metrics import roc_auc_score

# Predict probabilities (not 0/1)
y_probs = model.predict(X_test).ravel()  # Flatten just in case
auc_score = roc_auc_score(y_test, y_probs)
print("AUC:", auc_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
AUC: 0.45


In [127]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_hat))

[[18  2]
 [ 9  0]]
